In [49]:
from pprint import pprint
from paddlenlp import Taskflow
import paddle
import pandas as pd
import ast
from datetime import datetime

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


In [50]:
text = ''
# files_path = r'../../temp/甘肃省_学习考察.xlsx'
files_path = r'../../temp/全对测试集_学习考察.xlsx'
# paddle.device.set_device("gpu")

In [51]:
# original_data = pd.read_excel(files_path, sheet_name='Sheet2')
original_data = pd.read_excel(files_path)
data = original_data.copy()

In [52]:
def process_content(x):
    # 防止长度过长
    if len(x) > 1500:
        return x
    return ','.join(x)

data['content'] = data['content'].apply(ast.literal_eval)
data['content'] = data['content'].apply(process_content)

In [55]:
schema_list = ["政府领导到其他城市考察学习或访问", "不属于政府领导到其他城市考察学习的新闻","其他"]

# 创建分类器
cls = Taskflow("zero_shot_text_classification", schema=schema_list, batch_size=32, max_seq_len=1024, precision='fp32', model='utc-xbase')

In [56]:
loop_limit = 5

# 存储最终结果
final_results = []

for content in data['content']:
    label_counts = {label: 0 for label in schema_list}  # 用于记录每个label的出现次数
    label_total_scores  = {label: 0 for label in schema_list}  # 用于记录每个label的最高score
    
    for i in range(loop_limit):
        temp = cls(content)
        
        # 获取每一轮的预测结果
        predictions = temp[0].get('predictions', [])
        
        for prediction in predictions:
            label = prediction.get('label')
            score = prediction.get('score')
            
            # 增加label的出现次数
            label_counts[label] += 1
            
            # 更新最高的score
            label_total_scores[label] += score
                
    # 找出出现次数最多的label
    final_label = max(label_counts, key=label_counts.get)
    
    # 确保不会除以零
    if label_counts[final_label] > 0:
        final_score = label_total_scores[final_label] / label_counts[final_label]
    else:
        final_score = 0  # 或者设置为一个适当的默认值
    
    # 记录最终结果
    final_results.append({'content': content, 'label': final_label, 'score': final_score})

# 输出最终结果
for result in final_results:
     print(f"Content: {result['content']}, Label: {result['label']}, Score: {result['score']}")

In [59]:
df = pd.DataFrame(final_results)
# 获取当前时间并格式化为字符串
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
df.to_csv(f'{current_time}_temp.csv', index=False)

In [58]:
# text_types = [label.get('predictions') for label in temp]
# text_types